本来我是想看看这几只：
"510050","510310","510500","512000","512560","512600"他们的价格相关性，
后来挑了3只最不相关的一起投：
python -m dingtou.pyramid_v2.pyramid_v2 \
-c 510500,512000,512560 \
-s 20150101 \
-e 20230101 \
-b sh000001 \
-a 500000 \
-m -480 \
-gs 1000 \
-gh 0.01 \
-o 5
效果依然一般，不是说相关就一定会好。结论是。


In [2]:
import akshare as ak
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import talib
import numpy as np
from matplotlib.font_manager import FontProperties
import os
import mplfinance as mpf 
import importlib
import utils
importlib.reload(utils)

%matplotlib inline
# 添加字体
matplotlib.rcParams['axes.unicode_minus']=False # 如果不加入下面这一句，会导致负号无法显示
plt.rcParams["font.sans-serif"] = ["SimHei"]

In [32]:
fund_codes = ["510050","510310","510500","512000","512560","512600"]

dfs  = [utils.load_fund(code) for code in fund_codes]
dfs


[            close    code
 date                     
 2004-12-30  1.000  510050
 2004-12-31  0.996  510050
 2005-01-07  0.981  510050
 2005-01-14  0.983  510050
 2005-01-21  0.988  510050
 ...           ...     ...
 2022-12-12  3.907  510050
 2022-12-13  3.910  510050
 2022-12-14  3.927  510050
 2022-12-15  3.914  510050
 2022-12-16  3.929  510050
 
 [4364 rows x 2 columns],
              close    code
 date                      
 2013-03-06  1.0000  510310
 2013-03-08  1.0029  510310
 2013-03-15  1.0002  510310
 2013-03-18  0.9973  510310
 2013-03-19  1.0001  510310
 ...            ...     ...
 2022-12-12  1.8787  510310
 2022-12-13  1.8751  510310
 2022-12-14  1.8806  510310
 2022-12-15  1.8793  510310
 2022-12-16  1.8804  510310
 
 [2383 rows x 2 columns],
              close    code
 date                      
 2013-02-06  1.0000  510500
 2013-02-08  1.0022  510500
 2013-02-22  0.9954  510500
 2013-03-01  1.0135  510500
 2013-03-08  0.9980  510500
 ...            ...     ...
 2022

In [33]:
dfs = [df.rename(columns={'close':df.iloc[0].code}) for df in dfs]

dfs = [df.drop(columns=['code']) for df in dfs]

In [34]:
df = pd.concat(dfs, axis=1)

df.corr()

,510050,510310,510500,512000,512560,512600
510050,1.000000,0.979928,0.704234,0.574705,0.657738,0.886913
510310,0.979928,1.000000,0.780063,0.629944,0.827600,0.916963
510500,0.704234,0.780063,1.000000,0.722622,0.917152,0.390755
512000,0.574705,0.629944,0.722622,1.000000,0.586690,0.453899
512560,0.657738,0.827600,0.917152,0.586690,1.000000,0.845746
512600,0.886913,0.916963,0.390755,0.453899,0.845746,1.000000


In [23]:
dfs[0].drop(columns=['code'])

,close
date,
2013-02-06,1.0000
2013-02-08,1.0022
2013-02-22,0.9954
2013-03-01,1.0135
2013-03-08,0.9980
...,...
2022-12-12,1.9833
2022-12-13,1.9729
2022-12-14,1.9646
